In [1]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="whitegrid", context='notebook', color_codes=True)

from haversine import haversine

The goal of this notebook is convert our cleaned events dataset to a format that is suitable for classification algorithm.

Input Datasets:
- Cleaned Events Data: `data/events_no_duplicates.csv`
- Cleaned IATA Data: `data/iata_no_duplicates.csv`

Intuitions:
1. Users that search multiple times will mostly book a trip.
2. Users will most likely book when destination is far from their origin.
3. Users will most likely book based on the number of vacation days.

# Events Dataset

In [2]:
filepath = "data/events_no_duplicates.csv"
dataset = pd.read_csv(filepath)

In [3]:
dataset.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0


In [4]:
dataset['date_from'] = pd.to_datetime(dataset['date_from'])
dataset['date_to'] = pd.to_datetime(dataset['date_to'])
dataset['num_days'] = (dataset['date_to'] - dataset['date_from']).dt.days
dataset.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,num_days
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,6
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,21
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,3
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0,3
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0,6


# Calculating geo distance

I use haversine formula to calculate the distance between 2 places.

In [5]:
iata = pd.read_csv('data/iata_no_duplicates.csv').set_index('iata_code')

In [6]:
geo_distances = {}

def calculate_distance(x):
    origin = x['origin']
    destination = x['destination']
    key = f"{origin}-{destination}"
    
    if key not in geo_distances:   
        origin_geo = iata.loc[origin]
        origin_point = (origin_geo['lat'], origin_geo['lon'])
    
        dest_geo = iata.loc[destination]
        dest_point = (dest_geo['lat'], dest_geo['lon'])
        
        geo_distances[key] = haversine(origin_point, dest_point)
    
    return geo_distances[key] 

In [7]:
dataset['geo_distance'] = dataset.apply(lambda x: calculate_distance(x), axis=1)

In [8]:
dataset.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,num_days,geo_distance
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,6,5837.828026
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,21,6530.035012
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,3,470.077408
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0,3,1499.761224
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0,6,2923.178366


# Let's calculate how many searches before the user book a trip

In [18]:
dataset[dataset['user_id'] == '0035d4'].sort_values(by=['ts'])

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,num_days,geo_distance
5719,2017-04-24 02:20:28,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,2,0,12,1839.949396
6189,2017-04-24 02:29:04,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,2,0,12,1839.949396
333,2017-04-28 04:26:21,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396
9570,2017-04-28 04:46:36,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396
7278,2017-04-28 05:09:25,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396
8105,2017-04-29 03:20:11,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396
6175,2017-04-29 03:33:59,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396
176,2017-04-29 03:43:29,search,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396
8048,2017-04-29 03:48:22,book,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396


For example, user `0035d4` has 8 search events before it books a trip. I'll create a new dataset with new columns `search_count` and `is_booked`. So, the resulting data for `0035d4` would be the following:

In [19]:
pd.DataFrame(
    [{
        'user_id': '0035d4',
        'date_from': '2018-02-08',
        'date_to': '2018-02-20',
        'origin': 'DUS',
        'destination': 'AGP',
        'num_adults': 3,
        'num_children': 1,
        'num_days': 12,
        'geo_distance': 1839.949396,
        'search_count': 8,
        'is_booked': 1
    }]
)

,user_id,date_from,date_to,origin,destination,num_adults,num_children,num_days,geo_distance,search_count,is_booked
0,0035d4,2018-02-08,2018-02-20,DUS,AGP,3,1,12,1839.949396,8,1


In [20]:
def transform_dataset(dataset):
    """
    Transforms the dataset by grouping events with similar details and adding new columns ['search_count', 'is_booked']
    """
    
    columns = ['user_id', 'date_from', 'date_to', 'origin', 'destination']
    grouped_data = dataset.groupby(columns)

    new_dataset = []
    for index, group in grouped_data:
        obj = dict(zip(columns, index))
        booked_data = group[group['event_type'] == 'book']
        data = group.iloc[0]
    
        if not booked_data.empty:
            obj['is_booked'] = 1
            data = booked_data.iloc[0]
        else:
            obj['is_booked'] = 0
                        
        obj['search_count'] = group[group['event_type'] == 'search'].shape[0]
        obj['num_adults'] = data['num_adults']
        obj['num_children'] = data['num_children']
        obj['num_days'] = data['num_days']
        obj['geo_distance'] = data['geo_distance']
    
        new_dataset.append(obj)
    
    return pd.DataFrame(new_dataset)

In [21]:
grouped_dataset = transform_dataset(dataset)

In [23]:
grouped_dataset.head()

,user_id,date_from,date_to,origin,destination,is_booked,search_count,num_adults,num_children,num_days,geo_distance
0,000069,2017-05-10,2017-05-26,FRA,ALG,0,1,2,3,16,1553.717725
1,00019d,2017-09-15,2017-10-13,PAR,SSA,0,1,4,0,28,7944.570036
2,0002e3,2017-05-01,2017-05-01,PAR,ATH,0,1,1,0,0,2095.453304
3,000332,2017-06-08,2017-07-07,NCE,TLV,0,3,1,0,29,2722.329760
4,000332,2017-06-08,2017-07-07,TLV,NCE,0,1,2,0,29,2722.329760


Now, that we have our transformed data, let's store the new dataset to `data/transformed_dataset.csv`

In [24]:
grouped_dataset.to_csv('data/transformed_dataset.csv', index=False)